# E91 Implementation

Rory Schadler, Blake Danziger  
Physics 75  
Lab 2

## Protocol Steps
1. Alice and Bob share $n$ EPR pairs.
1. Alice picks a random measurement basis $a_i$ from $a = \{0, \frac{\pi}{8}, \frac{\pi}{4}\}$ and Bob picks a random measurement basis $b_j$ from $b = \{-\frac{\pi}{8}, 0, \frac{\pi}{8}\}$ where $a_i$ and $b_j$ represent the angle of rotation $\theta$ between the measurement basis and the $Z$(computational) basis (not to be confused with the angle on the Bloch Sphere. The angle between the $z$-axis and the axis of the measurement basis on the Bloch Sphere is $2\theta$). Alice and Bob measure each of their respective qubits of the EPR pairs in a randomly chosen measurement basis from their set of bases.
1. Alice and Bob anounce the measurement bases they used for each shared pair. Bits for which their bases were the same are saved into respective bit strings $m_a$ and $m_b$.
1. The measurement bits in $m_a$ and $m_b$ should be anticorrelated between Alice and Bob, so Bob flips the bits in $m_b$ to get the same bit string as $m_a$. This string of bits is the shared key.
1. For the bits where they used different measurement bases, Alice and Bob announce the results of their measurements. These bits can be used to check for the presence of Eve

## Implementation Steps
1. Create $n$ Bell pairs in the $\frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)$ state.
1. Choose measurement bases for Alice and Bob randomly from each of their sets of bases $a = \{0, \frac{\pi}{8}, \frac{\pi}{4}\}$ and $b = \{-\frac{\pi}{8}, 0, \frac{\pi}{8}\}$.
1. Measure each qubit in the pair
    * For a basis with angle $\theta$, rotate about the $y$-axis using a $U3$ gate with polar rotation angle $\theta$. ($\phi = \lambda = 0$ in order to simulate a $RY$ gate)
1. Find the pairs where Alice and Bob used the same measurement basis and save these bits after flipping Bob's bits.
1. Compare measurements of bits where Alice's and Bob's measurements did not match

Source: [https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.67.661](https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.67.661)

### Import Libraries

In [68]:
import qiskit
from qiskit import QuantumCircuit, Aer, assemble
from qiskit.visualization import plot_histogram
from numpy.random import randint#, choice
import numpy as np
print("Imports Successful")

Imports Successful


### Set Message Length

In [69]:
msg_len = 10

### Define measurement bases

In [70]:
a_b = [np.pi/8, 0, np.pi/4]
b_b = [np.pi/8, 0, -np.pi/8]

### Prepare Entangled EPR States

In [71]:
# prepare entangled states as separate 2-qubit quantum circuits

def prepare_EPR(msg_len):
    # initialize output
    pairs = []
    
    # set up (1/sqrt(2))*(|01> - |10>) states
    for i in range(msg_len):
        qc = QuantumCircuit(2,2)
        qc.x(0)
        qc.x(1)
        qc.h(0)
        qc.cx(0,1)
        qc.barrier()
        
        pairs.append(qc)
    return pairs

### Choose Random Bases and Measure Qubits in Chosen Bases

In [72]:
def measure(pairs):
    
    # set up measurement simulation
    backend = Aer.get_backend('qasm_simulator')
    
    measurements = []
    alice_bases = []
    bob_bases = []
    
    # for each EPR pair
    for q in range(len(pairs)):
        
        # pick random basis for Alice's qubit
        alice_bases.append(randint(3))
        bob_bases.append(randint(3))
        
        alice_rot = a_b[alice_bases[q]]
        bob_rot = a_b[bob_bases[q]]
        
        # measure qubits in chosen bases by rotating about the y-axis at the opposite of the angle and then measuring
        pairs[q].u3(-alice_rot, 0, 0, 0)
        pairs[q].u3(-bob_rot, 0, 0, 1)
        
        pairs[q].measure(0,0)
        pairs[q].measure(1,1)
        
#         display(pairs[q].draw())
        
        
        
        
        qasm_sim = Aer.get_backend('qasm_simulator')
        qobj = assemble(pairs[q], shots=1, memory=True)
        result = qasm_sim.run(qobj).result()
        measured_bit = str(result.get_memory()[0])
        measurements.append(measured_bit)
        
    return (alice_bases, bob_bases, measurements)

### Check Measured Results

In [73]:
def check(alice_bases, bob_bases):
    for idx, (a_basis, b_basis) in enumerate(zip(alice_bases, bob_bases)):
        if (a_basis == b_basis) and (a_basis != 2):
            a_m = m[idx][0]
            b_m = m[idx][1]
            print("match! idx {}, Alice measured {} and Bob measured {}".format(idx, a_m, b_m))
            if a_m != b_m:
#                 print("good match!")
                pass
            else:
                print("bad match. idx {}, a_b {}, a_m {}, b_b {}, b_m {}".format(idx, a_b, a_m, b_b, b_m))

In [74]:
def run_e91(msg_len):
    pairs = prepare_EPR(msg_len)
    (a_b, b_b, m) = measure(pairs)
    
    check(a_b, b_b)

    print(a_b)
    print(b_b)
    print(m)
    
    return m

In [75]:
for i in range(10):
    run_e91(10)

match! idx 6, Alice measured 0 and Bob measured 1
[0, 2, 0, 2, 2, 1, 0, 1, 0, 2]
[2, 2, 1, 0, 0, 0, 0, 2, 1, 2]
['01', '01', '01', '01', '01', '10', '10', '10', '10', '10']
match! idx 9, Alice measured 1 and Bob measured 0
[0, 2, 1, 2, 0, 1, 0, 2, 1, 1]
[1, 0, 0, 0, 1, 0, 2, 2, 0, 1]
['10', '10', '01', '10', '01', '10', '01', '10', '01', '10']
match! idx 5, Alice measured 0 and Bob measured 1
match! idx 9, Alice measured 1 and Bob measured 0
[2, 1, 1, 1, 2, 1, 0, 0, 1, 0]
[0, 2, 0, 0, 2, 1, 2, 2, 0, 0]
['10', '10', '10', '10', '01', '10', '01', '01', '01', '01']
match! idx 0, Alice measured 0 and Bob measured 1
match! idx 5, Alice measured 0 and Bob measured 1
match! idx 8, Alice measured 1 and Bob measured 0
[0, 0, 0, 0, 2, 1, 0, 0, 1, 2]
[0, 2, 2, 1, 1, 1, 2, 1, 1, 0]
['10', '10', '10', '10', '10', '01', '01', '10', '01', '10']
match! idx 0, Alice measured 0 and Bob measured 1
match! idx 4, Alice measured 1 and Bob measured 0
[1, 2, 2, 0, 0, 1, 1, 0, 2, 1]
[1, 1, 1, 1, 0, 0, 0, 2, 2,

In [76]:
import qiskit.tools.jupyter
%qiskit_version_table